# 🛍️ 產品推薦系統 - Google Colab 示範

這個 Notebook 展示如何在 Google Colab 上運行產品推薦系統。

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ldsAS/product-recommendation-system/blob/main/notebooks/colab_demo.ipynb)

## 📋 功能展示

1. 安裝依賴套件
2. 克隆專案
3. 準備示範資料
4. 訓練推薦模型
5. 獲取產品推薦
6. 視覺化結果

## 1️⃣ 環境設置

In [ ]:
# 檢查 Python 版本
import sys
print(f"Python 版本: {sys.version}")

# 安裝必要套件
!pip install -q pandas numpy scikit-learn lightgbm pydantic pydantic-settings python-dotenv

## 2️⃣ 克隆專案

In [ ]:
# 克隆 GitHub 倉庫
!git clone https://github.com/ldsAS/product-recommendation-system.git
%cd product-recommendation-system

# 顯示專案結構
!ls -la

## 3️⃣ 準備示範資料

由於真實資料較大，我們創建一些示範資料來展示系統功能。

In [ ]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

# 設定隨機種子
np.random.seed(42)

# 創建示範會員資料
n_members = 100
members = pd.DataFrame({
    '會員編號': [f'CU{i:06d}' for i in range(1, n_members + 1)],
    '電話': [f'09{np.random.randint(10000000, 99999999)}' for _ in range(n_members)],
    '總消費金額': np.random.randint(1000, 100000, n_members),
    '累積紅利': np.random.randint(0, 5000, n_members),
    '註冊日期': [(datetime.now() - timedelta(days=np.random.randint(30, 730))).strftime('%Y-%m-%d') for _ in range(n_members)]
})

# 創建示範產品資料
n_products = 50
products = [f'{i:05d}' for i in range(30000, 30000 + n_products)]

# 創建示範銷售資料
n_sales = 500
sales = pd.DataFrame({
    '訂單編號': [f'S{i:06d}' for i in range(1, n_sales + 1)],
    '會員編號': np.random.choice(members['會員編號'], n_sales),
    '訂單日期': [(datetime.now() - timedelta(days=np.random.randint(1, 365))).strftime('%Y-%m-%d') for _ in range(n_sales)],
    '訂單金額': np.random.randint(100, 5000, n_sales),
    '門市代碼': np.random.choice(['STORE01', 'STORE02', 'STORE03'], n_sales)
})

# 創建示範訂單明細
salesdetails = []
for order_id in sales['訂單編號']:
    n_items = np.random.randint(1, 5)
    for i in range(n_items):
        salesdetails.append({
            '訂單編號': order_id,
            '產品編號': np.random.choice(products),
            '數量': np.random.randint(1, 5),
            '單價': np.random.randint(50, 1000),
            '小計': np.random.randint(50, 5000)
        })
salesdetails = pd.DataFrame(salesdetails)

# 儲存示範資料
members.to_csv('data/raw/member', index=False)
sales.to_csv('data/raw/sales', index=False)
salesdetails.to_csv('data/raw/salesdetails', index=False)

print("✅ 示範資料已創建")
print(f"會員數: {len(members)}")
print(f"訂單數: {len(sales)}")
print(f"訂單明細數: {len(salesdetails)}")

## 4️⃣ 資料探索

In [ ]:
# 查看會員資料
print("📊 會員資料樣本:")
display(members.head())

print("\n📊 銷售資料樣本:")
display(sales.head())

print("\n📊 訂單明細樣本:")
display(salesdetails.head())

## 5️⃣ 訓練推薦模型

In [ ]:
# 設置環境變數和路徑
import os
import sys

# 設置工作目錄
os.chdir('/content/product-recommendation-system')

# 添加專案路徑到 Python 路徑
sys.path.insert(0, '/content/product-recommendation-system')

# 設置 PYTHONPATH 環境變數
os.environ['PYTHONPATH'] = '/content/product-recommendation-system'

print("✅ 環境設置完成")
print(f"當前目錄: {os.getcwd()}")
print(f"Python 路徑: {sys.path[0]}")

In [ ]:
# 執行訓練（使用 PYTHONPATH）
print("🚀 開始訓練推薦模型...")
print("=" * 60)

!cd /content/product-recommendation-system && PYTHONPATH=/content/product-recommendation-system python src/train.py

print("\n" + "=" * 60)
print("✅ 訓練完成！")

# 檢查模型是否生成
import os
model_dir = '/content/product-recommendation-system/data/models'
if os.path.exists(model_dir):
    models = [f for f in os.listdir(model_dir) if f.endswith('.pkl')]
    if models:
        print(f"\n📦 找到 {len(models)} 個模型檔案:")
        for model in models:
            print(f"  - {model}")
    else:
        print("\n⚠️  警告: 未找到模型檔案")
else:
    print("\n⚠️  警告: 模型目錄不存在")

## 6️⃣ 測試推薦功能

In [ ]:
# 確保路徑正確
import sys
import os

os.chdir('/content/product-recommendation-system')
if '/content/product-recommendation-system' not in sys.path:
    sys.path.insert(0, '/content/product-recommendation-system')

# 檢查模型是否存在
model_dir = 'data/models'
if not os.path.exists(model_dir):
    print("❌ 錯誤: 模型目錄不存在")
    print("請先執行步驟 5 訓練模型")
else:
    model_files = [f for f in os.listdir(model_dir) if f.endswith('.pkl')]
    if not model_files:
        print("❌ 錯誤: 未找到模型檔案")
        print("請先執行步驟 5 訓練模型")
    else:
        print(f"✅ 找到模型: {model_files[0]}")
        
        # 導入必要模組
        try:
            from src.models.recommendation_engine import RecommendationEngine
            from src.models.data_models import RecommendationRequest
            
            # 初始化推薦引擎
            engine = RecommendationEngine()
            
            # 選擇一個測試會員
            test_member = members.iloc[0]
            
            # 創建推薦請求
            request = RecommendationRequest(
                member_code=test_member['會員編號'],
                phone=test_member['電話'],
                total_consumption=float(test_member['總消費金額']),
                accumulated_bonus=float(test_member['累積紅利']),
                recent_purchases=[]
            )
            
            print(f"\n🎯 為會員 {test_member['會員編號']} 生成推薦...")
            print(f"總消費: ${test_member['總消費金額']:,}")
            print(f"累積紅利: {test_member['累積紅利']} 點")
            
            # 獲取推薦
            recommendations = engine.get_recommendations(request)
            
            print("\n✨ 推薦結果:")
            print("=" * 60)
            
            for i, rec in enumerate(recommendations.recommendations, 1):
                print(f"\n{i}. 產品編號: {rec.product_id}")
                print(f"   信心分數: {rec.score:.2%}")
                print(f"   推薦理由: {rec.reason}")
                if rec.explanation:
                    print(f"   詳細說明: {rec.explanation}")
            
            print("\n" + "=" * 60)
            print(f"模型版本: {recommendations.model_version}")
            print(f"回應時間: {recommendations.response_time_ms:.0f} ms")
            print("\n✅ 推薦成功！")
            
        except Exception as e:
            print(f"\n❌ 推薦失敗: {e}")
            print("\n除錯資訊:")
            import traceback
            traceback.print_exc()
            print("\n💡 提示:")
            print("1. 確認已執行步驟 5 訓練模型")
            print("2. 檢查模型檔案是否存在於 data/models/")
            print("3. 嘗試重新執行步驟 5")

## 7️⃣ 批次測試多個會員

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 設定視覺化風格
sns.set_style('whitegrid')
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'DejaVu Sans']

# 測試前 10 個會員
test_results = []

for idx in range(min(10, len(members))):
    member = members.iloc[idx]
    request = RecommendationRequest(
        member_code=member['會員編號'],
        phone=member['電話'],
        total_consumption=float(member['總消費金額']),
        accumulated_bonus=float(member['累積紅利']),
        recent_purchases=[]
    )
    
    try:
        result = engine.get_recommendations(request)
        test_results.append({
            'member_code': member['會員編號'],
            'num_recommendations': len(result.recommendations),
            'avg_score': np.mean([r.score for r in result.recommendations]),
            'response_time': result.response_time_ms
        })
    except:
        pass

if test_results:
    results_df = pd.DataFrame(test_results)
    
    # 視覺化結果
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # 平均信心分數
    axes[0].bar(range(len(results_df)), results_df['avg_score'])
    axes[0].set_xlabel('Member Index')
    axes[0].set_ylabel('Average Confidence Score')
    axes[0].set_title('Average Recommendation Confidence')
    axes[0].set_ylim([0, 1])
    
    # 回應時間
    axes[1].bar(range(len(results_df)), results_df['response_time'])
    axes[1].set_xlabel('Member Index')
    axes[1].set_ylabel('Response Time (ms)')
    axes[1].set_title('Recommendation Response Time')
    axes[1].axhline(y=3000, color='r', linestyle='--', label='3s Target')
    axes[1].legend()
    
    plt.tight_layout()
    plt.show()
    
    print("\n📊 測試統計:")
    print(f"平均信心分數: {results_df['avg_score'].mean():.2%}")
    print(f"平均回應時間: {results_df['response_time'].mean():.0f} ms")
    print(f"最大回應時間: {results_df['response_time'].max():.0f} ms")
else:
    print("❌ 無法生成測試結果")

## 8️⃣ 模型資訊

In [ ]:
# 顯示模型資訊
model_info = engine.get_model_info()

print("🤖 模型資訊:")
print("=" * 60)
print(f"模型版本: {model_info.model_version}")
print(f"模型類型: {model_info.model_type}")
print(f"訓練時間: {model_info.trained_at}")
print(f"\n效能指標:")
for key, value in model_info.metrics.items():
    print(f"  {key}: {value}")
print("=" * 60)

## 9️⃣ 互動式推薦測試

In [ ]:
# 互動式輸入
from IPython.display import display, HTML

def test_recommendation_interactive():
    print("🎮 互動式推薦測試")
    print("=" * 60)
    
    # 隨機選擇一個會員或讓使用者輸入
    member_idx = int(input(f"請輸入會員索引 (0-{len(members)-1}): ") or "0")
    member = members.iloc[member_idx]
    
    print(f"\n選擇的會員: {member['會員編號']}")
    print(f"總消費: ${member['總消費金額']:,}")
    print(f"累積紅利: {member['累積紅利']} 點")
    
    request = RecommendationRequest(
        member_code=member['會員編號'],
        phone=member['電話'],
        total_consumption=float(member['總消費金額']),
        accumulated_bonus=float(member['累積紅利']),
        recent_purchases=[]
    )
    
    recommendations = engine.get_recommendations(request)
    
    print("\n✨ 推薦結果:")
    for i, rec in enumerate(recommendations.recommendations, 1):
        print(f"\n{i}. {rec.product_id} (信心度: {rec.score:.2%})")
        print(f"   {rec.reason}")

# 執行互動式測試
# test_recommendation_interactive()  # 取消註解以執行

## 🎉 完成！

您已成功在 Google Colab 上運行產品推薦系統！

### 下一步

1. **使用真實資料**: 上傳您自己的資料檔案到 Colab
2. **調整參數**: 修改模型參數以優化效能
3. **部署到生產**: 參考 [DEPLOYMENT.md](../docs/DEPLOYMENT.md)

### 相關資源

- [GitHub 倉庫](https://github.com/ldsAS/product-recommendation-system)
- [完整文檔](../README.md)
- [API 文檔](../docs/API_GUIDE.md)
- [模型訓練指南](../docs/MODEL_TRAINING.md)